### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.stg_marts_tables.dim_product_forecast"  ,
        "desafioadventureworks-446600.stg_marts_tables.dim_product_seasonality",
        "desafioadventureworks-446600.stg_staging_tables.stg_production_product",
        "desafioadventureworks-446600.stg_staging_tables.stg_sales_store"

    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.stg_marts_tables"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [ ]:
# Configuração para que o df exiba todas as colunas e todas as linhas completas, e também, exiba o formato numérico com 2 dígitos após a vírgula

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela com substituição de '-' por '_'
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    # Ler os dados da tabela do BigQuery para um df
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    # Armazenar o df no dicionário
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Print de validação
print("Todas as tabelas foram processadas com sucesso!")


In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
dim_product_forecast = df_processados['dim_product_forecast']

print(f"Colunas: {dim_product_forecast.shape[1]}\nLinhas: {dim_product_forecast.shape[0]}")

In [ ]:
# Ordenar e exibir o df por 'productid_id'
dim_product_forecast = dim_product_forecast.sort_values(by=['productid_id'])

print(dim_product_forecast)

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in dim_product_forecast.columns:   
    missing_rows = dim_product_forecast[dim_product_forecast[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = dim_product_forecast.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

#doc: currentflag possue somente 1 valor, o que indica que pode ser somente valores True ou False.

In [ ]:
#verificar informações do df
dim_product_forecast.info()

In [ ]:
#Visão Geral da Previsão: 

# Total de demanda por loja*

demand_by_store = dim_product_forecast.groupby('store_id')['forecast_quantity'].sum().reset_index()
demand_by_store = demand_by_store.sort_values(by='forecast_quantity', ascending=False)

print(demand_by_store.head(15))


#doc*: soma da previsão de demanda agrupada por loja


# Conclusões sobre a previsão de demanda por loja:

# 1. A loja com o ID 4 possui a maior previsão de demanda (17,850,928.00), indicando ser a principal 
#    consumidora de produtos nos próximos 3 meses. Isso sugere que ela deve receber maior atenção
#    na distribuição de produtos e planejamento de estoque.

# 2. A loja com o ID 6 é a segunda maior em previsão de demanda (14,796,192.00), também se destacando
#    como um centro importante para alocação de produtos.

# 3. As lojas com IDs 1, 10 e 2 completam o top 5, apresentando demandas significativamente menores,
#    mas ainda consideráveis. Estas lojas podem ser consideradas secundárias em termos de prioridade.

# 4. A diferença entre as lojas com maior (ID 4) e menor (ID 8) demanda no top 10 é de aproximadamente
#    15,184,704 unidades. Isso mostra uma concentração de demanda em certas lojas.

# 5. O padrão de demanda evidencia que os esforços logísticos e de distribuição devem ser direcionados
#    para lojas de alta demanda (principalmente IDs 4 e 6), enquanto as demais podem ser monitoradas
#    com menos frequência.


In [ ]:
# Gráfico ajustado de barras para lojas sem valores no eixo Y e com valores no topo das barras

sns.set(style="white")  # Fundo branco
plt.figure(figsize=(10, 6))  # Ajustar o tamanho do gráfico

sns.barplot(data=demand_by_store, x='store_id', y='forecast_quantity', palette='Blues_d')
plt.title('Demanda Prevista por Loja')
plt.xlabel('Loja (Store ID)')
plt.ylabel('')  # Remover o rótulo do eixo Y

# Remover valores do eixo Y
plt.gca().yaxis.set_ticks([])

# Adicionar valores no topo das barras com uma casa decimal
for index, row in demand_by_store.iterrows():
    plt.text(
        x=index, 
        y=row['forecast_quantity'], 
        s=f"{row['forecast_quantity']/1000000:.1f}M", 
        ha='center', 
        va='bottom', 
        fontsize=10, 
        color='black'
    )

plt.show()


In [ ]:
#Total de demanda por produto*:

demand_by_product = dim_product_forecast.groupby('productid_id')['forecast_quantity'].sum().reset_index()
demand_by_product = demand_by_product.sort_values(by='forecast_quantity', ascending=False)

print(demand_by_product.head(10))




# Conclusões sobre a previsão de demanda por produto:

# 1. O produto com ID 712 possui a maior previsão de demanda (3,996,816.00), indicando que ele será o mais requisitado
#    entre todos os produtos nos próximos 3 meses. Este produto deve ser priorizado na produção e reposição.

# 2. O produto com ID 715 é o segundo em previsão de demanda (3,291,088.00), seguido pelos IDs 711 (3,199,136.00),
#    708 (3,066,240.00) e 707 (2,890,224.00). Esses produtos compõem o top 5 em demanda prevista.

# 3. A diferença entre o produto mais demandado (ID 712) e o quinto mais demandado (ID 707) é de aproximadamente
#    1,106,592.00 unidades, indicando uma concentração de demanda significativa nos produtos mais procurados.

# 4. A presença de produtos com alta demanda relativamente próxima indica que esses itens precisam ser monitorados
#    como um grupo prioritário para ajustes em produção e estoque, dependendo das vendas reais.

# 5. Produtos fora do top 5, como o ID 714 (1,729,872.00), ainda apresentam volumes consideráveis de demanda,
#    mas podem ser considerados de prioridade secundária em um cenário de otimização de recursos.

#doc: os dados são baseados em previsões de demanda nos próximos 3 meses, agrupados por produto.




In [ ]:
# Ajustar estilo do gráfico
sns.set(style="white")  # Fundo branco
plt.figure(figsize=(10, 6))  # Ajustar o tamanho do gráfico

# Selecionar apenas os Top 10 produtos com maior demanda
top_products = demand_by_product.sort_values(by='forecast_quantity', ascending=False).head(10)

# Gráfico de barras verticais para produtos com ordenação
ax = sns.barplot(
    data=top_products, 
    x='productid_id', 
    y='forecast_quantity', 
    palette='Blues_d',
    order=top_products['productid_id']
)

plt.title('Top 10 Produtos com Maior Demanda Prevista')
plt.xlabel('Produto (Product ID)')
plt.ylabel('')  # Remover o rótulo do eixo Y

# Remover valores do eixo Y
plt.gca().yaxis.set_ticks([])

# Adicionar valores no topo das barras com uma casa decimal
for bar in ax.patches:
    ax.text(
        x=bar.get_x() + bar.get_width() / 2, 
        y=bar.get_height(), 
        s=f"{bar.get_height() / 1000000:.3f}M", 
        ha='center', 
        va='bottom', 
        fontsize=10, 
        color='black'
    )

plt.show()


## Sazonalidade - Produto ID == 712

In [ ]:
# Atribuir o df a uma variável com nome mais simples
dim_product_seasonality = df_processados['dim_product_seasonality']

print(f"Colunas: {dim_product_seasonality.shape[1]}\nLinhas: {dim_product_seasonality.shape[0]}")

In [ ]:
dim_product_seasonality.head()

In [ ]:
# Atribuir o df a uma variável com nome mais simples
stg_production_product = df_processados['stg_production_product']

print(f"Colunas: {stg_production_product.shape[1]}\nLinhas: {stg_production_product.shape[0]}")

In [ ]:
# Fazer o merge para adicionar o nome do produto
dim_product_seasonality = dim_product_seasonality.merge(
    stg_production_product, 
    how='left', 
    on='productid_id'
)

# Garantir que o produto com ID 712 está correto
product_name = dim_product_seasonality[dim_product_seasonality['productid_id'] == 712]['product_nm'].iloc[0]


In [ ]:
# Filtrar os dados para o produto 712
filtered_product = dim_product_seasonality[dim_product_seasonality['productid_id'] == 712]

# Obter o nome do produto (garantir que ele existe no merge)
product_name = filtered_product['product_nm'].iloc[0] if not filtered_product.empty else "Desconhecido"

# Criar uma coluna 'sale_month' para exibição no gráfico
filtered_product['sale_month'] = filtered_product['year'].astype(str) + '-' + filtered_product['month'].astype(str).str.zfill(2)

# Agrupar por 'sale_month' e somar a quantidade total vendida
monthly_sales = filtered_product.groupby('sale_month')['total_quantity_sold'].sum().reset_index()

# Ordenar os meses para garantir a sequência correta
monthly_sales = monthly_sales.sort_values(by='sale_month')

# Visualizar os dados em um gráfico de linhas com valores nos marcadores
plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_sales, x='sale_month', y='total_quantity_sold', marker='o')

plt.title(f'Variação Mensal de Vendas - {product_name} (ID: 712)')
plt.xlabel('Mês (Ano-Mês)')
plt.ylabel('Quantidade Vendida')
plt.xticks(rotation=45)  # Rotacionar os rótulos para melhor leitura
plt.grid(visible=True, linestyle='--', alpha=0.5)

# Adicionar os valores nos marcadores
for i, row in monthly_sales.iterrows():
    plt.text(
        x=row['sale_month'], 
        y=row['total_quantity_sold'] + 15,  # Ajustar a posição acima do marcador
        s=row['total_quantity_sold'], 
        ha='center', 
        fontsize=9, 
        color='black'
    )

plt.show()

In [ ]:
# Conclusões sobre o gráfico de sazonalidade:

# 1. O produto analisado é o "AWC LOGO CAP" (ID: 712).
#    Este produto apresenta uma variação considerável na quantidade de vendas ao longo dos meses.

# 2. Períodos de alta:
#    - Observa-se um pico significativo de vendas em maio de 2013, alcançando 663 unidades.
#    - Outros meses de destaque incluem junho de 2012 e junho de 2013, que também apresentam volumes elevados.

# 3. Períodos de baixa:
#    - Os meses de dezembro de 2011 e fevereiro de 2014 têm os volumes mais baixos, com 25 e 93 unidades vendidas, respectivamente.
#    - É possível observar quedas acentuadas após os picos, indicando uma sazonalidade cíclica.

# 4. Tendência sazonal:
#    - O padrão de vendas sugere um comportamento sazonal com picos frequentes em meses próximos ao meio do ano (maio e junho).
#    - Esses padrões podem estar relacionados a eventos sazonais, promoções ou alta demanda típica nesses períodos.

# 5. Implicações para o planejamento:
#    - A sazonalidade identificada indica que a produção e o estoque devem ser aumentados nos meses de alta demanda (maio-junho).
#    - Para os meses de baixa demanda, um ajuste no planejamento de produção e distribuição pode reduzir custos e evitar excesso de estoque.


# Questão 11

In [ ]:
# agora a questão 11:

# Um novo fornecedor de luvas, que agora engloba toda a produção mundial, precisa ter uma estimativa de quantos zíperes precisa pedir para os próximos 3 meses. Levando em consideração que são necessários 2 zíperes por par, quantos seriam necessários?

# A abordagem será a seguinte:

# Filtrar os produtos de luvas: Identificar quais registros correspondem a luvas no dataset. Para isso, usaremos a coluna product_nm ou uma característica relevante.
# Prever a demanda futura para luvas: Utilizar o modelo para calcular a demanda para os próximos 3 meses, assim como foi feito para a demanda geral.
# Calcular o total de zíperes necessários: Multiplicar a demanda de luvas por 2 (já que são necessários 2 zíperes por par).
# Exibir o resultado: Retornar o valor total de zíperes.
# Vamos começar? Se tiver alguma dúvida ou ajuste, é só avisar!

In [ ]:
# Listar todas as variáveis definidas no ambiente atual (corrigido)
variables_snapshot = list(globals().items())  # Criar uma cópia estática de globals()

for var_name, var_value in variables_snapshot:
    if isinstance(var_value, pd.DataFrame):
        print(f"DataFrame: {var_name} - Dimensão: {var_value.shape}")


In [ ]:
# Visualizar os primeiros registros do DataFrame stg_production_product
print("\nVisualizando os primeiros registros de stg_production_product:")
print(stg_production_product.head())
print("\nInformações sobre stg_production_product:")
print(stg_production_product.info())




In [ ]:
# Sim, o stg_production_product parece ser o DataFrame ideal para obter informações sobre os produtos, incluindo os relacionados a luvas, considerando que contém detalhes como product_nm (nome do produto) e productnumber_cd (código do produto). Além disso, ele já possui informações relevantes para análise, como:

# productid_id: Identificador único do produto.
# product_nm: Nome do produto, essencial para identificar luvas.
# listprice_vr: Preço listado.
# safetystocklevel_nr e reorderpoint_tp: Dados úteis para estimativas relacionadas ao estoque.
# Próximos passos:
# Filtrar os produtos que correspondem a luvas (product_nm contendo "GLOVES").
# Cruzar essas informações com o dim_product_forecast para prever a demanda dos próximos 3 meses.
# Usar a previsão para calcular a quantidade de zíperes necessária.
# Quer que eu prepare o script para filtrar os produtos "GLOVES" no stg_production_product?

In [ ]:
# Filtrar produtos relacionados a "GLOVES" no stg_production_product
gloves_products = stg_production_product[stg_production_product['product_nm'].str.contains("GLOVES", case=False, na=False)]

# Visualizar os primeiros registros dos produtos filtrados
print("Produtos filtrados contendo 'GLOVES':")
print(gloves_products[['productid_id', 'product_nm', 'listprice_vr']].head(10))

# Verificar a dimensão do dataset filtrado
print(f"Dimensão do dataset filtrado: {gloves_products.shape}")


In [ ]:
# Ótimo, já temos os produtos relacionados a "GLOVES" filtrados. Agora podemos cruzar essa informação com o dataset dim_product_forecast para obter a previsão de demanda desses produtos nos próximos 3 meses.

In [ ]:
# Visualizar os primeiros registros do DataFrame dim_product_forecast
print("\nVisualizando os primeiros registros de dim_product_forecast:")
print(dim_product_forecast.head())
print("\nInformações sobre dim_product_forecast:")
print(dim_product_forecast.info())




In [ ]:
print(dim_product_forecast.head())

In [33]:
# Filtrar apenas as previsões dos produtos "GLOVES"
gloves_forecast = dim_product_forecast[dim_product_forecast['productid_id'].isin(gloves_products['productid_id'])]
print("Produtos filtrados (GLOVES):")
print(gloves_forecast.head())




Produtos filtrados (GLOVES):
        store_id  productid_id forecast_date  forecast_quantity
72916          7           858    2025-01-07             740.00
27670          3           858    2025-02-07             396.00
87178          8           858    2025-03-07             660.00
102403        10           858    2025-02-07             704.00
75479          7           858    2025-02-07             740.00


In [34]:
# Converter a coluna 'forecast_date' para o tipo datetime
gloves_forecast['forecast_date'] = pd.to_datetime(gloves_forecast['forecast_date'])
print("\nDatas convertidas para datetime:")
print(gloves_forecast[['productid_id', 'forecast_date']].head())



Datas convertidas para datetime:
        productid_id forecast_date
72916            858    2025-01-07
27670            858    2025-02-07
87178            858    2025-03-07
102403           858    2025-02-07
75479            858    2025-02-07


C:\Users\wrpen\AppData\Local\Temp\ipykernel_14232\929107084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast['forecast_date'] = pd.to_datetime(gloves_forecast['forecast_date'])


In [35]:

# Obter a data mínima para identificar o ponto de partida
start_date = gloves_forecast['forecast_date'].min()
print(f"\nData mínima (início do período): {start_date}")




Data mínima (início do período): 2025-01-07 00:00:00


In [42]:
from dateutil.relativedelta import relativedelta

# Recalcular 'month_diff' com base em meses reais
gloves_forecast['month_diff'] = gloves_forecast['forecast_date'].apply(
    lambda x: relativedelta(x, start_date).months + 1
)
print("\nColuna 'month_diff' criada com meses reais:")
print(gloves_forecast[['productid_id', 'forecast_date', 'month_diff']].head())

# Criar as colunas 'month_1', 'month_2', 'month_3' com base no novo cálculo
for month in [1, 2, 3]:
    gloves_forecast[f'month_{month}'] = gloves_forecast.apply(
        lambda x: x['forecast_quantity'] if x['month_diff'] == month else 0, axis=1
    )
    print(f"\nPreenchendo os dados para 'month_{month}':")
    print(gloves_forecast[[f'month_{month}', 'forecast_quantity', 'month_diff']].head())



Coluna 'month_diff' criada com meses reais:
        productid_id forecast_date  month_diff
72916            858    2025-01-07           1
27670            858    2025-02-07           2
87178            858    2025-03-07           3
102403           858    2025-02-07           2
75479            858    2025-02-07           2

Preenchendo os dados para 'month_1':
        month_1  forecast_quantity  month_diff
72916    740.00             740.00           1
27670      0.00             396.00           2
87178      0.00             660.00           3
102403     0.00             704.00           2
75479      0.00             740.00           2

Preenchendo os dados para 'month_2':
        month_2  forecast_quantity  month_diff
72916      0.00             740.00           1
27670    396.00             396.00           2
87178      0.00             660.00           3
102403   704.00             704.00           2
75479    740.00             740.00           2

Preenchendo os dados para 'month

C:\Users\wrpen\AppData\Local\Temp\ipykernel_14232\3604093273.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast['month_diff'] = gloves_forecast['forecast_date'].apply(
C:\Users\wrpen\AppData\Local\Temp\ipykernel_14232\3604093273.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast[f'month_{month}'] = gloves_forecast.apply(
C:\Users\wrpen\AppData\Local\Temp\ipykernel_14232\3604093273.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [43]:
print(gloves_forecast[gloves_forecast['month_diff'] == 3])


       store_id  productid_id forecast_date  forecast_quantity  month_diff  month_1  month_2  month_3  total_demand
87178         8           858    2025-03-07             660.00           3     0.00     0.00   660.00        660.00
16459         2           858    2025-03-07             688.00           3     0.00     0.00   688.00        688.00
14107         2           858    2025-03-07             688.00           3     0.00     0.00   688.00        688.00
16458         2           858    2025-03-07             688.00           3     0.00     0.00   688.00        688.00
76160         7           858    2025-03-07             740.00           3     0.00     0.00   740.00        740.00
...         ...           ...           ...                ...         ...      ...      ...      ...           ...
37710         4           863    2025-03-07            2824.00           3     0.00     0.00  2824.00       2824.00
15117         2           863    2025-03-07            1484.00          

In [44]:
print(gloves_forecast['forecast_date'].min(), gloves_forecast['forecast_date'].max())


2025-01-07 00:00:00 2025-04-07 00:00:00


In [45]:
# Calcular a demanda total de luvas para os próximos 3 meses
gloves_forecast['total_demand'] = gloves_forecast[['month_1', 'month_2', 'month_3']].sum(axis=1)
print("\nDemanda total calculada para cada produto (linhas de exemplo):")
print(gloves_forecast[['productid_id', 'month_1', 'month_2', 'month_3', 'total_demand']].head())




Demanda total calculada para cada produto (linhas de exemplo):
        productid_id  month_1  month_2  month_3  total_demand
72916            858   740.00     0.00     0.00        740.00
27670            858     0.00   396.00     0.00        396.00
87178            858     0.00     0.00   660.00        660.00
102403           858     0.00   704.00     0.00        704.00
75479            858     0.00   740.00     0.00        740.00


C:\Users\wrpen\AppData\Local\Temp\ipykernel_14232\3407250140.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast['total_demand'] = gloves_forecast[['month_1', 'month_2', 'month_3']].sum(axis=1)


In [46]:
# Verificar se 'total_demand' está correto
gloves_forecast['check_total'] = gloves_forecast[['month_1', 'month_2', 'month_3']].sum(axis=1)
print("\nValidação de 'total_demand' com 'check_total':")
print(gloves_forecast[['productid_id', 'total_demand', 'check_total']].head())



Validação de 'total_demand' com 'check_total':
        productid_id  total_demand  check_total
72916            858        740.00       740.00
27670            858        396.00       396.00
87178            858        660.00       660.00
102403           858        704.00       704.00
75479            858        740.00       740.00


C:\Users\wrpen\AppData\Local\Temp\ipykernel_14232\3266386074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gloves_forecast['check_total'] = gloves_forecast[['month_1', 'month_2', 'month_3']].sum(axis=1)


In [47]:
# Somar a demanda total de todos os produtos "GLOVES"
total_gloves_demand = gloves_forecast['total_demand'].sum()
print(f"\nDemanda total de luvas nos próximos 3 meses: {total_gloves_demand}")




Demanda total de luvas nos próximos 3 meses: 2608824.0


In [48]:
# Calcular a quantidade de zíperes necessários (2 zíperes por par de luvas)
total_zippers_needed = total_gloves_demand * 2
print(f"Quantidade total de zíperes necessários: {total_zippers_needed}")

Quantidade total de zíperes necessários: 5217648.0


In [ ]:
# Validar se o cálculo de 'total_demand' está correto
# O valor de 'total_demand' deve ser igual à soma de 'month_1', 'month_2' e 'month_3' para cada linha
# A validação mostra que os valores estão consistentes.

# Demanda total de luvas para os próximos 3 meses
# A soma de 'total_demand' em todo o dataset representa o total de luvas previstas para os próximos 3 meses
# Resultado: 2.608.824 luvas

# Calcular a quantidade total de zíperes necessários
# Cada par de luvas utiliza 2 zíperes, então multiplicamos a demanda total de luvas por 2
# Resultado: 5.217.648 zíperes necessários

# Conclusão:
# A análise confirma que a demanda prevista para luvas nos próximos 3 meses foi corretamente somada,
# e a quantidade de zíperes necessários foi calculada com base nessa previsão.


In [ ]:
# Contexto da análise
# Justificativa:
# - O objetivo da análise foi determinar a quantidade de zíperes necessária para atender à demanda de luvas
#   nos próximos 3 meses, conforme solicitado pelo fornecedor global.
# - Considerando que cada par de luvas exige 2 zíperes, é fundamental que as previsões estejam alinhadas com
#   a demanda de luvas para evitar problemas na cadeia de suprimentos, como falta de estoque ou desperdício.

# Escopo do que foi feito:
# 1. Filtragem dos dados:
#    - Identificamos os produtos classificados como "GLOVES" no dataset `stg_production_product`.
#    - Filtramos a tabela de previsões (`dim_product_forecast`) para incluir apenas os IDs dos produtos de luvas.
# 2. Criação de colunas mensais:
#    - Calculamos as previsões de demanda para os próximos 3 meses, organizando os dados em colunas 
#      'month_1', 'month_2' e 'month_3'.
#    - Somamos essas colunas para obter a demanda total de luvas no período.
# 3. Cálculo da demanda total:
#    - Consolidamos a previsão de demanda total para todos os produtos de luvas nos próximos 3 meses.
#    - Multiplicamos a demanda total por 2 (zippers por par de luvas) para determinar a quantidade de zíperes necessária.
# 4. Validação:
#    - Verificamos se os cálculos estão consistentes ao comparar as somas das colunas mensais com a coluna de demanda total.

# Resultado:
# - Demanda total prevista de luvas: 2.608.824 pares.
# - Quantidade total de zíperes necessária: 5.217.648 unidades.
